# Explicit References

Lettuce so far .. has no side effects. 


## Side effects
    -> Printing
    -> Sending a packet
    -> Updating a global var / static global member of a class
    
The main thing is that the issues after the side effects are caused. 

Pure functions are when no matter how many times you do something, the answers will be the same 

```
    7 + 5 = 15
    7 + 5 = 15
    7 + 5 = 15...
    
    
but consider this code in C

int x // global variable

int foo() {
    x = x+1 
    return x 
}

// main
x = 0; // set the global variable
foo(); // return a value
foo(); 
foo();

at this point we see that our global x is now 3, the function
does not behave the same everytime
```

Side effects are very common, without them we couldn't do some important things such as updating a global variable / static global member of a class


                Examples of references. 
__Var in scala__,  in scala this is an implicit reference

__Ptr in C/C++__,  this is an explicit reference, its the ADDRESS

Todo: Add explicit reference rules in Lettuce. 

## Lettuce With Explicit References

__Concrete Syntax:__

```
Program 1

    let x = newref(15) 
        in 
            deref(x) 
            
            essentially 'newref' creates new memory in the 'store'
            
Everytime you say newref, a new cell is created in the store and the item inside of newref is stored inside that address. 

x is now a reference (or a pointer) to the address of 15!

deref(x) , the deref essentially says "go into that address and give me what is in there". 
    so deref(x) returns the numValue(15). 
```

now lets look at a more complicated problem 


```
Program 2: 
    let x = newref(15) in 
        let y = newref(17) in 
            let z = assignref(x, 25) in 
                deref(x) + deref(y) 
                    // program returns 42. 
                
Here in this program we had created 2 memory cells in x and y in the store containing 15, 17. So x and y are now references to their memory address storing their numbers. 

What is assignref? Read it as (*x = 25) , its the equivalent to *x, so z gets the result of the assignment. 

it takes the contents of the memory and assigns it to 25, so the address at reference x NOW CONTAINS numValue 25!

so we don't actually care about what z is, whereas for the most part we just used it to reassign x. 
```

```
Program 3. 

    let incr = function (x) 
                assign(x, deref(x) + 1
           in  
       let z = newref(0) in 
           let y1 = incr (z) in 
               let y2 = incr(2) in 
                   let y3 = incr(z) in 
                       deref(z) 
                       
So essentially, we just kept going into the memory address of the
original z and ketp changing it by incrememnting it by 1. 
```

So, here are the new additions to the lettuce syntax. 

__newRef(...Expr...)__ , this is suppose to create a new cell in the memory and put the inner value in the cell. 

__deref(..Expr..)__, take a reference to cell in memory and retrieve its contents. 

__assignref(expr1, expr2)__, reference to the memory you are assigning to and the value you are assigning to that cell. 


So now we have our grammar. 

Expr => Const(double)
        Ident(String
        IfThenElse(Expr,Expr,Expr)
        Let(String,Expr,Expr)
        FunDef(String, Expr) 
        LetRec(String, String, Expr, Expr) 
        FunCall(Expr,Expr) 
        NewRef(Expr)
        DeRef(Expr) 
        AssignRef(Expr,Expr) 
        

Let("x", NewRef(Const(15)), 
    Let("y", NewRef(Const(17)), 
        Let("z", assignRef(Ident("x", Const(25)), 
            Plus(Deref(Ident("x"), Deref(Ident("y")))
            
Wew have now introduced the overall idea of explicit references!

What is the difference between let bindings and these new references. 


```
        let x = 15 in
            let y = 20 in 
                let z = 45 in 
                    x + y + z
            all this is saying is that whenever we see any of the variables in the programs, replace them with their assignment. 
```

# Lecture Stuff

```
    let x = newRef(15) in        , x is just a reference to address 0 in the 'store', NewRef(Const(15))
        let y = deref(x) + 20 in        Deref(Ident("x"))
            let z = assignref(x,y) in      AssignRef(Ident("x"), Ident("y")) 
                deref(x) 
                
             essentially this program just created spot in memory, x, then changed it to the value of y. 
             
& are not permitted in our program 
    
    let *x = &p 
    
Pointer arithmetic is also not allowed

    int *z = y + 15
```

## Abstract Syntax: Lettuce with Explicit References. 

    Expr => Const(Double) 
            Ident(String) 
            Plus(Expr, Expr) 
            ...
            Let(String, Expr, Expr) 
            FunDef(...)
            FunCall(...)
            LetRec(...) 
            
            NewRef(Expr) 
            DeRef(Expr)    , this expression better be a reference. 
            AssignRef(Expr, Expr) 
            
            
We must make some adjustments to our value types. 

Value Types, so far we have
               Value => NumValue(Double) 
                        BoolValue(Boolean) 
                        Closure(String, Expr, Environment) 
                        Error
                        
                        // this is the new stuff
                        Reference(Int)  // integer is going to be a memory address. 
                        
A store is a linear set 

## Reworking evaluatation steps

    eval(e: Expr, env: Environment, x: Store): (Value, Store) 


1. Creation of a new cell
    
    newCell(s: Store, v: Value) : (Store, Int) 
        this function will return a new store with a new address added for the value and the new address created. 
        
2.  getContents(s: Store, j: Int) : Value


3.  setContents(s: Store, j: Int, v: Value) : Store


## The new semantics for eval. 

Signature for eval have changed. 
    
       eval(e: Expr, env: Environment, s: Store): (Value, Store)
       
       
```
    Consider the following program 
    
        let n = newref(10) 
            in 
                (assignRef(x, 25) + dref(n) 
                
                this program will work, but it has side effects, it will just do 25 + 25 
```


    


## Notes on Implicit-References-Vars

### Implicit References 
Previously we, looked at references using statements such as NewRef, DeRef, and AssignRef. 

Our goal in this notebook is to mimic the behavior of mutable vars in scala. 

In [1]:
var x = 10
val y = x 
x = x + 1
print(s"The value of x is $x")

The value of x is 11

x: Int = 11
y: Int = 10

In this lecture we will study __implicit references__. 
    * These are references but we do not want to use NewRef to create a reference, we will look at syntax similar to 'var' declarations in scala
    * We do not wish to use derefs to get value. Whenever we refer toa var, we would directly like its value without using a deref 
    * Finally, we would like to use assignment on these vars just like we did on references
    * Finally, we would like to use assignment on these vars like we did on references
    

### Syntax for Vars in Lettuce 

Let us add a bit of extra syntax that will allow us to create such implicit references. Since this is lettuce, we will use let var bidning to specify that whatever is being bound to an implicit reference. 

```
    let var x = <expr> in <expr> 
```

We will discard explicit reference operations NewRef, AssignRef, DeRef. Instead, we will just have the let var binidng to create new implicit references (we will call them "vars" since they will behave exactly like scala's vars), and assignments. 

Let us do some example programs. 

```
    let var x = 10 in 
        let dummy = AssignVar(x, 20) in 
            x 
            
This will be equivalent to 

    let x = NewRef(10) in 
        let dummy = AssignRef(x, 20) in 
            DeRef(x)
```

Not that AssignRef is now called AssignVar, there is no more DeRef. These are two major changes in our syntax. 

What does this program do

```
    let var x = 10 in 
        let g = function(y)
            x```
           in 
         let dummy = AssignVar(x, 20) in 
             g(dummy) 
             
We can implement some funky stuff such as changing the binding of the function. 

    let var f = function (x) x + 10 in 
        let g = function(y) y - 10 in 
            let d = f(10) in 
                let dummy = AssignVar(f, g) 
                    in 
                        d - f(10) 
                        
Notice how we assigned f to g because f is a var. So it is assignable. Also not that we do not need to say DeRef in this langauge. Finally, notice how g is an immutable val whereas f is a mutable var
```

## Abstract Syntax of Lettuce With Mutable References

__Program__ --> TopLevel(__Expr__) 
__Expr__ --> All the other grammars...
            | LetVar(Identifer, Expr, Expr) 
            | AssignVar(Identifier, Expr)   assign Var to a value
            

## Operational Semantics
How do we evaluate implicit references under the hood? Simple, the same way as we evaluate explicit references. In other words, we are going to keep the innards of our interpreter unchanged from what it was previously 


Let us recap how this was done. This part is cut and paste from the last week's notes. 
    1. We will makea new value type for references since our expressions an evaluate to real numbers, booleans, closures and now   references. 
        2. To go hand in hand with references, we need to define an abstract notion of of memory. We will call this a store. 
        
        Note that the vars will point to these references
        
## Stores

Memory address are going to be numbered 0,1,2 ... with natrual numbers and each address is going to be associated with a value. 

The store needs to support the following operations. 
    * Create a new memory cell in the store and assign a value to it. This is exactly what will implement the NewRef operation. Let us call it "createNewCell" opeartion on stores. 
    * Lookup the value of memory cell. Let us call it "lookupCellVlaue" operation. If this value does not exist, we will return error (and bail) 
    * Assign a cell to a new vale. Let us call it assignToCell
    
Hand in hand with stores, we need to extend our vlau etype. Existing value types are 

    * ErrorValue - denotes and error, we just throw an exception 
    * NumValue(f) for number f, denoted as all REAL numbers
    * BoolValue(b) for booleans
    * Closure(x , e , sigma) for closures. We will denote these as Closure in our semantics. 
    
    
Finally, we add references: 
    * Reference(j) , which is a reference to a cell number j in our store. 
    
## Operational Semantics
Once again, our opeartional semantics defines a eval function that has three parts to it. 

       eval(expr, env, store) = value, new-store) 
       
Let us explain how it will be organized. There are two kinds of bindings: 
    * immutable vals are bound to values in the environment
    * mutable vars are bound to references(j) where j is an address in the store. 
    
## Semantic Rule for Constant

This rule does not change

$$ \semRule{}{\eval(\texttt{Const(f)}, \sigma, s) = ( f, s) }{const} $$



## Semantic Rule for Identifiers (How automatic Deref is implemented) 

Identifiers are going to be slightly more complex. Let us go over the first interesting case. The following rule ensures that whenever we evaluate an identifer x that happens to be a var, we will actually return the stored value

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j), \texttt{lookupCell}(s, j) = v}{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-var-ok}$$


* The variable x belongs to the environment SIGMA 
* it evaluates to a reference to cell j in store
* cell j in store s has value v
* Evaluating the identifer x under environment SIGMA and store s has value v


Note how that whenever we touch a reference, we automatically chase the value corresponding to that reference in the store and return that. 

For immutables, we have the usual semantics: 


## Semantic Rules for LetVar
We will now write the semantic rule for let ref. The idea is that we will generate a new reference. 

$$\semRule{\eval(\texttt{e1}, \sigma, s) = (v, s_1), \;\; v \not= \mathbf{error},\;\;\; \texttt{createNewCell}(s_1,v) = (j, s_2)\;\;  }{\eval(\texttt{LetVar(x, e1, e2)}, \sigma, s) =  \eval(\texttt{e2}, \sigma[x \mapsto \texttt{Reference}(j)], s_2 }{let-var-ok}$$

* First evaluate e1 under the environment SIGMA and store s, results in value v that is not error and store s1. 
* create a new cell in s1, let it be a reference to cell number j in s2
* evaluating let var x = e1 in e2 is the same as evaluating d2 under environemt sigma[x -> reference(j)] and store s2


## Semantic Rule for AssignVar
The semantic rule for AssignVar is the same as that for AssignRef but under a new guise. 

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j),\;\;\eval(\texttt{e}, \sigma, s) = (v, s_1), \;\;  \texttt{assignToCell}(s_1, j, v) = s_2}{ \eval(\texttt{AssignVar(x, e)}, \sigma, s) = (v, s_2) } {assign-var-ok} $$

* x must be ampped to a reference to cell j in the current envionrment sigma
* e must evaluate under sigma and store s to v with new store s1
* the store s2 is obtained when we assign the value v to cell j in s1
* the expression AssignVar(x,e) under env sigma and store s yields value v and the store s2 

The remaining rules remain unchanged 

# Function calls : call by value call by reference


With the introduction to explicit and implicit references, we will need to dive a little deper into how function calls are handled and understood the key concepts of call by value vs orther calling conventions. Many lagnauges provide both calling conventions in some form and this is something we need to understand in the context of Lettuce with mutables. 

As a result of this lecutre, you will be able to understand some confusing aspects of langauges like Scala (and python) behave. 

In [2]:
// Program A 
def foo(x: Int): Int = {
    if (x % 2 == 1)
        3 +x + 1
    else{
        x/2
    }
}

/* Function call first evals the arg, then passes its "Value" 
    into the arugment of foo
*/
val z = 10  // immutable z 
foo(10)

defined function foo
z: Int = 10
res1_2: Int = 5

The program above should return the value 5. It is very easy to reason why. WHen we call foo(x), the value of x is bound to 10. The function foo is called withthe arugment 10 and rest is easy to reason about. 

In [2]:
// Program B
def bar (x: Int) : Int = { // paremeters to function calls are vals 
    x = 25 // assigns to 25
    2 * x
}

var z = 12 // multable z 
/* function evals the arg(z) which is a return of the reference of
what ever z is holding, so 12 */ 
bar(z) // mutables z's VALUE is being passed to bar, not the reference so this code doesn't work

cmd2.sc:2: reassignment to val
    x = 25 // assigns to 25
      ^Compilation Failed

: 

The program above delcares a __mutable__ variable z, assigns it to 12. When we try to pass it to the function bar as x and assigns it to 25 in of x. It does not work. The program complains that x is a val and cannot be resassigned. Let us try the same thing but in a different context

In [3]:
// Program C
case class Wrapper(var x: Int) // field x is a mutable 

def bar(z: Wrapper) : Int = {
    z.x = 25 // assigns z.x to 25 
    2 * z.x // returns 2 * z.x
}


/* Here you just created a wrapper holding a VAR X = 0 
    so in essence you did essentially pass a reference to bar 
*/
val w = Wrapper(0)
println("Before call to bar --> w.x = " + w.x)
bar(w) // w is being "passed by value" 
println("After call to bar --> w.x = " + w.x)

Before call to bar --> w.x = 0
After call to bar --> w.x = 25


defined class Wrapper
defined function bar
w: Wrapper = Wrapper(25)
res2_4: Int = 50

Something very funny is happening her: 
    * In program B, we passed a mutable integer intoa function bar but it refused to reassign it and complains that we cannot reassign it. 
    * In program B, we wrap the mutable around a class wrapper and declare it/pass an instance of wrapper w as a val but it not only allows us to reassign but also updates the value of w.x
    
We will undertand this once we study things in the simple case of Lettuce    

# Call By Value

Call by value measn taht whenever we call a function on an arguement, 
    funcCalled ( arguement ) 
    
the arugment is (fully) evaluated and its value is passed into the function call

Consider the lettuce program: 
```
    let x = 10 in 
        let f = function (z) 
            2 * z in 
               f( x + 20 ) 
```


* First eval the argument: It evaluates to the number 30
* Next, call the function on the value of the arguemnt
    * The formal parameter z = now 30 
    * Function returns 60 
    
Call by value is straightforward to reason about once you understand this principle. What will the foolowing Lettuce program evaluate to? 

```
    let bar = function(x) 
        let dummy1 = assignref(x, 25) in 
            2 * dereft(x) 
            
        in 
        let z = newref(12) in 
            bar(z) 
```

Once again there is a function called bar(z). The rule for call by value is to evaluate the argument. 
    * The value of arugment z is reference(0): A reference to memory cell 0, which has the vlaue 12 in it. 
    * The code executes and assigns to reference that was passed in (now bound to the formal parameter z) 
    * as a result, the ocntents of the cell 0 in thememory are updated to 25 
    * the result of the prgram is 50
    
In other words, call by value is able to achieve side effects when the value being passed in the function is a pointer 


## Scala always does call by value as default

Scala supports two basic conventions: Call yb value and somewhat esoteric calling conventions such as "call by name" and "call by need" (using call by name and lazy values)

```
let us go back to program B above (recalled here)

def bar(x: Int): Int = {
    x = 25 
    2 * x
}

var z = 12 
bar(z) 
```
It fails for two reasons: 
    * The parameters for functions in scala are always vals and thus immutbale. Thus the parameter x for function bar in the code above is an immutable and therfore it is not reassignable. 
    * The vale of z in the call to bar(z) is 12, and is passed directly as a NumValue(12) 
    
There are langauges where this program would be excepted but still not work (python) 

In the lettuce langauge with implicit references, we still do call by value. 

    * we evaluate the arguement to bar(z) 
        * ident("z") in the samentics of ident, evaluates not toa reference but to the actual contents of the memory, which is 12. 
        * the call to abr fails ebcasue, we try to do assignVar on an identifier x that is not a reference. 
        
How do we then expalin the program of Program C (recalled below):
```
case class Wrapper(var x: Int)

def bar(z: Wrapper): Int ={
    // z = wrapper(1) 
    z.x = 25 
    2 * z.x
}

val w = Wrapper(0)
println("Before call to bar -- > w.x = " + w.x)
bar(w)
println("After call to bar -- > w.x = " + w.x)
```

Once again, scala will do a call by value. Therefore the call to bar(w) passes the object w we just created by value

    * However, the value of an object (something declared as a calss or object) in scala is a pointer / reference to the object. Therefore, w is passed in as some kidn of a Reference( ... wherever w is stored ..) . As a rseult we are allowed to accses z.x in the program which is a var and is in fact the very same thing as w.x form outside of the function call. 
    
    
## Why special treatment for objects?
Langauges like Scala(and java, and python etc) treat objects differently from basic types such as ints. The basic difference is that the value of an Int in the context of "call by value" is itself, whereas the value ascribed to an object in "call by value" is not a copy of the object but a reference to the contents of the object. You can see the difference in this program directly

In [4]:
class Wrapper(var x: Int)

def bar(w: Wrapper): Int = {
    println("Inside bar: Passed in value w = " + w)
    w.x
}

def foo(x: Int): Int = {
    println("Inside foo: passed in x = " + x)
    x
}

val x1 = bar(new Wrapper(10))
val x2 = foo(20)



Inside bar: Passed in value w = ammonite.$sess.cmd3$Helper$Wrapper@27bc7817
Inside foo: passed in x = 20


defined class Wrapper
defined function bar
defined function foo
x1: Int = 10
x2: Int = 20

Languages like C and C++ do not do this kind of special treatment when you ask them to call by value they WILL make a copy of everything and call by value. This can be quite expensive. 


# Call by Reference

In call by reference smenatics, the value ofth eparameters are notpassed but rather the parameters are passed as references. This was orginallaly implemented in fortran. Let us take an example in "fortran" - like syntax

```
    SUBROUTINE SWAP( X, Y) 
        TMP = X
        X = Y
        Y = TMP
    A = 45
    B = 55
    SWAP (A, B) 
```

Here, we define a "subroutine" called SWAP with two arguments, X and Y: it exchanges their values. In a regular call by value semantics, this program has no effect on A and B outside function call. This is because , in call by value, SWAP is called on the numbers 45 and 55. In call by reference, the call to the swap function SWAP maps the parameter X to A and Y to B

Therefore the assignments that happen to X, Y insdie the functoin reflect on the variables A, B in the caller of the function

C++ supports call by reference by placing an & in front of the parameter in the functino definiont 

```
void swap( &x, &y){
    int temp = x 
    x = y
    y = temp
}

void main(){
    int a = 45; 
    int b = 55; 
    swap(a,b) // thought you had to put *a, *b
    // values are now swapped. 
}
```

Call by reference is useful because it allows the following advantages
    * Allow "return parameters" which are assigned inside the function as multiple return values. 
    * Allow lower cost of passing of large data structures since they will perform a copy. 
    
Going back to the previous example, we can write the function foo with call by reference. This stops the compiler from performing a copy of the entire BigDataStruct contents when foo is called. 


# Continuation Passing Style

Thus far, we have built interpretters for various features in Lettuce. However, all of our interpreters depended on recurisive calls to the eval function . The use of revursion was very convenient for us to translate smeantics directly intoa scala program. However, this is ont idael since we all know about recursion and stacks. Thus, alrge programs can cause the stack to overflow

In this lecutr,e we wil revist the theme of eliminating non-tail recursion. We have already done this using an accumulator. However, accumulators are limited in their scope. We will now prseenta. general scheme that works without accumulators. 

## Recap: Recursion, Tail Recursion and Eliminating the Non-Tail Recusion

1. Recursion causes the acivation of records to grow on the stack, potentially causing a stack overflow. 
2. Tail recursion is a benign case when the result of any recursive call are returned without any further processing
3. Tail recursive calls can be implemented such that the activations need not to grow. 


In [5]:
/*-- Examples: As an exercise, classify these calls as tail recursive or not --*/

def rec_fun1(x: Int): Int = {// yes
    if (x <= 0) {
        x
    } else {
        rec_fun1(x - 10)
    }
}

def rec_fun2(y: Int = 0, x: Int): Int = {
    if (x <= 10) {
        y
    } else {
        rec_fun2(y + 1, x - 10)  // yes 
    }
}


def rec_fun3( x: Int): Int = { // no 
    if (x <= 10) {
        x - 10
    } else {
        1 + rec_fun3( x - 10)
    }
}


def rec_fun4( x: Int): Int = {
    if (x <= 10) {
        x - 5
    } else {
        rec_fun1 ( rec_fun4( x - 10) ) // no 
    }
}

def foo(x: Int): Int = { x - 15}

def rec_fun5(x: Int): Int = { //yes
    if (x <= 0){
        foo(x)
    } else {
        rec_fun5(foo(x))
    }
}


def rec_fun6(x: Int): Int = {  // no 
    if (x <= 0){
        foo(x)
    } else {
        foo(rec_fun6(x-5))
    }
}

defined function rec_fun1
defined function rec_fun2
defined function rec_fun3
defined function rec_fun4
defined function foo
defined function rec_fun5
defined function rec_fun6

# Continuations and Continuation Passing Style
Continuation passing style (CPS) is a "style" of programming wherin every function willhave an extra argument called the continaution. A continuation function that is passed in and specifies what the caller wishes to do with the result that has been computed. 

Take for instance, a function func that takes in an integer and returns an integer

```
    def func(x: Int): Int = {
        // .. do some work to comput result ..
        return result
    }
    
In the CPS, this function will now be written as 

    def func0k(x: Int, k: Int => Int): Int = {
       // do some work to get the result 
       k(result) // pass the result on to the continuation 
    }
```

Note that funk-k takes in an extra argument k called continuation. It is the function throw which the called specifies what they awnt done with the result of the call. Rather than return the result and make the caller operate on it, the caller simply bundles up the results and passes it all in 

Lets look at a concrete example. First take a look at these three functions defined below

In [6]:
def addUp(x: Int, y: Int, z: Int): Int = {
    x + y + z 
}

def multiply(x: Int, y: Int): Int ={
    x * y 
}

def madd(x: Int, y: Int , z: Int): Int =  {
    val v1 = multiply(x,y) 
    val v2 = addUp(v1, y, z) 
    return v2
}

defined function addUp
defined function multiply
defined function madd

In [7]:
println(madd(1,2,3))

7


Let us now create the CPS version of these functions 

In [8]:
def addUp_k(x: Int, y: Int, z:Int, k: Int => Int): Int = {
    k(x + y + z)
}

def multiply_k(x: Int, y: Int, k: Int => Int): Int = {
    k ( x * y)
}

def madd_k(x: Int, y: Int, z: Int, k: Int => Int): Int ={
    // Create a new continuation.
    // This continuation k1 is a closure that will be passed to multiply.
    // It will be called by addUp_k but must do the work that was originally done by madd.
    def k1(v1: Int): Int = addUp_k(v1, y, z, k) // Call addUp on v1, y, z and ask addUp_k to run k on the result.
    multiply_k(x, y, k1)
}


defined function addUp_k
defined function multiply_k
defined function madd_k

In [9]:
madd_k(1,2,3, x=>x) // the function x => x is the identity function that just returns the argument

res8: Int = 7

A few things to notice: 
    * First, the translation for addUp and multiply to addUp_ka nd multiply_k is straightforward. These functions get a new arugment k for the continuation. They simply compute what they did orignally and instead of returning the result, they simply call k on it. 
    * However, the tricky function is madd_k function. What did the madd function do? 
    
        * called up addUp on x, y 
        * took the result and called the addUp function 
   * Thus , we can write down that madd_k should do
       * call mulitply_k on x,y and pass a continuation k1 to mulitply_k. What must this continuation do? 
       * The continuation k1 should do the arrears work madd would have done after the call to multiply returned
       
           1. call addUp_k
           2. Pass the result on to k

In [10]:
def f1(x: Int): Int = {
    if (x <= 0){
        1
    }
    else{
        3 + f1(x - 10)
    }
}

def f1_k(x: Int, k: Int => Int): Int = {
    if (x <= 0){
        k(1)
    }
    else{
        def k1(v: Int): Int = k(3 + v) // tell what to do with the result of the call
        f1_k(x - 10, k1) 
    }
}

defined function f1
defined function f1_k

In [11]:
println(f1(25))
println(f1_k(25, x => x))

10
10


In [12]:
import scala.annotation.tailrec
def factorial(n: Int) : Int = {
    if (n <= 0){
        1
    }else{
        n * factorial(n-1) 
    }
}

@tailrec
def factorial_k(n: Int, k: Int => Int): Int = {
    if (n <= 0){
        k(1)
    }else{
        def k1(v: Int): Int = {
            k(v * n)
        }
        factorial_k(n, k1) 
    }
}

import scala.annotation.tailrec

defined function factorial
defined function factorial_k

Note that the CPS style has important properties that you should check: 
    1. Every function f_k now has an extra argument called the continuation. 
    2. Teh input to the continuation is the result of the funciton f, output the type is same as input tpy.e 
    3. the original call has a "terminal continuation" typically x => x 
    4. each path in the code ends in a function call and there cannot be any intermiediate funciton call. 
    5. __ all of these calls are tail calls __ 
    
In theory therefore, these tail calls must be optimaized away by the compiler/interpreter. However, in practice it may not always be as simple. For instance, Scala has serious limitaiton on how it handles tail calls.     

In [13]:
def fibonacci(n: Int): Int = {
    if (n < 2){
        1
    } else {
        fibonacci(n-1) + fibonacci(n-2)
    }    
}


def fibonacci_k(n: Int, k: Int => Int): Int = {
    if (n < 2) {
        k(1)
    } else {
        // k1 is the continuation that will be passed to 
        // fibonacci(n-1). It instructs that call on what to do 
        // afterwards.
        def k1(v: Int): Int = {
            // k2 is the continuation that will be passed to 
            // fibonacci(n-2). It instructs that call on what to do 
            // afterwards.
            def k2(v2: Int): Int = {
                k (v + v2) // Just add up the two fibonacci results and pass it to k.
            }
            fibonacci_k(n-2, k2) // Call fibonacci on n-2 and execute function k2 on the result
        }
        // call fibonacci on n-1 and execute k1 on the result.
        fibonacci_k(n-1, k1)
    }
}

defined function fibonacci
defined function fibonacci_k

In [14]:
println(fibonacci(12))
println(fibonacci_k(12, x => x))

233
233


In [15]:
println(fibonacci_k(18, x => x))

: 

In [15]:
import scala.annotation.tailrec

    
@tailrec
def fibonacci_k(n: Int, k: Int => Int): Int = {
    if (n < 2) {
        k(1)
    } else {
        fibonacci_k(n-1, {
            v1 => fibonacci_k(n-2, {
                v2 => k(v1+v2)
            })
        })
    }
}


cmd15.sc:10: could not optimize @tailrec annotated method fibonacci_k: it contains a recursive call not in tail position
            v1 => fibonacci_k(n-2, {
                             ^Compilation Failed

: 

# Recipe for transforming to CPS passing style. 
    * Add continuation parameter to all routines. What should the type of the parameter be? same as output
    * now transform each function in the CPS style. 
    
## Case 1
If the function we are converting has no other function calls, then there is not much to do. Just remember to call the continuation parameter k on the return value. 

In [16]:
def simple_fun(x: Int): Int = {
    val y = x * x
    val z = y + y - 5 * x 
    if (z <= 0){
        1
    }else{
        z
    }
}

def simple_fun_k(x: Int, k: Int => Int): Int = {
    val y = x * x
    val z = y + y - 5 * x
    if (z <= 0){
        k(1) // remember to call k on the return value
    }
    else{
        k(z)  // remember to call k on the return value
    }
}

defined function simple_fun
defined function simple_fun_k

## Case 2

If the function we are converting has just one function call in each branch and it is a tail call, then again, we simply convert the tail calls into their CPS version and remember to pass the continuation parameter to them. 

In [17]:
def tail_call_fun(x: Int): Int = {
    if ( x >= 0){
        simple_fun(x+y)
    }else{
        val y = x * x - 2
        simple_fun(y)
    }
}

def tail_call_fun_k(x: Int, k: Int => Int): Int = {
    if ( x >= 0){
        simple_fun_k(x + 1, k) // conver to CPS version and remember and remember to pass my own continuation in 
    }else{
        val y = x * x - 2
        simple_fun_k(y, k) 
    }
}

defined function tail_call_fun
defined function tail_call_fun_k

## Case 3
where we have to continue processing the data 

In [23]:
def fancy_function(x: Int, y: Int): Int = {
    if (x == 0)
        return 0
    else if (x > 0) {
        val s1 = 25
        val y1 = x * y + x - y
        s1 + y1
    } else {
        val y1 = tail_call_fun(x)
        y1 + y - 2 * x
    }
    
}

defined function fancy_function

In [24]:
def fancy_function_k(x: Int, y: Int, k: Int => Int): Int = {
    if (x == 0)
        return k(0)
    else if (x > 0) {
        val s1 = 25
        val y1 = x * y + x - y
        k(s1 + y1)
    } else {
        // Transform code after call 
        //  y1 + y - 2 * x
        def k1(y1: Int): Int = {
            k(y1 + y - 2 * x)
        }
        tail_call_fun_k(x,  k1)
    }
    
}

defined function fancy_function_k

In [25]:
def even_more_fancy(x: Int): Int = {
    val v1 = fancy_function(x, x - 2)
    val v2 = fancy_function(x-2, x)
    val v3 = tail_call_fun(v1)
    val v4 = v1 + v2 + v3
    fancy_function(v4, v3)
}


def even_more_fancy_k(x: Int, k: Int => Int): Int = {
    def k1 (v1: Int) : Int = {
        /* CPS TRANSFORM OF 
        val v2 = fancy_function(x-2, x)
        val v3 = tail_call_fun(v1)
        val v4 = v1 + v2 + v3
        fancy_function(v4, v3)
        */
        def k2(v2: Int): Int = {
            /* CPS TRANSFORM 
             val v3 = tail_call_fun(v1)
             val v4 = v1 + v2 + v3
            fancy_function(v4, v3)
            */
            def k3(v3: Int): Int = {
                /*
                 val v4 = v1 + v2 + v3
                fancy_function(v4, v3)
                */
                val v4 = v1 + v2 + v3
                k(fancy_function(v4, v3))
            }
            tail_call_fun_k(v1, k3)
            
        }
        fancy_function_k(x-2, x, k2)
    }
    
    fancy_function_k(x, x-2, k1)  
}

defined function even_more_fancy
defined function even_more_fancy_k

In [26]:
even_more_fancy(15)
even_more_fancy_k(15, x=>x)

res25_0: Int = -1498352559
res25_1: Int = 1124682442

# Polymorphic Continuations

Thus far, we have lived in the happy and lucky world where all functions had integer arguments and returned integers. Reality knocks (you down) and htus, we have contended with functions having a lot of possible return types. However, unfortunately, this means CPS transformation will have to chage return type of each function we are transforming. Also, this type is not known in advance. We will motivate the need to perform polymorphic continuations. 

Consider the following example

In [27]:
def utilityFunction(x: Int): Int = x + 2

def call1(x: String): String = (utilityFunction(x.toInt)).toString

def call2(x: Int): Float = utilityFunction(x).toFloat

def mainFunction(x: Int):String = {
    val v1 = call1(x.toString)
    val v2 = call2(x)
    v1 + v2.toString
}

defined function utilityFunction
defined function call1
defined function call2
defined function mainFunction

In [27]:
def utilityFunction_k(x: Int, k: Int => Int): Int = k(x + 2)

def call1_k(x: String, k: String=> String): String = {
    utilityFunction_k(x .toInt, { v => k(v.toString)})
}

def call2_k(x: Int, k: Float => Float): Float = {
    utilityFunction_k(x, {f => k(f.toFloat)})
}


cmd27.sc:4: type mismatch;
 found   : String
 required: Int
    utilityFunction_k(x .toInt, { v => k(v.toString)})
                                        ^cmd27.sc:8: type mismatch;
 found   : Float
 required: Int
    utilityFunction_k(x, {f => k(f.toFloat)})
                                ^Compilation Failed

: 

The reason is that utilityFunciton_k is being called from two different call sites. Ufortunately, the continuations at these return sites return two different types. Therefore, we have to allow utilityFunction_k to be more general 

In [28]:
def utilityFunction_k[T1](x: Int, k: Int => T1): T1 = k(x + 2)

def call1_k[T2](x: String, k: String=> T2): T2 = {
    utilityFunction_k[T2]( x.toInt, { v => k(v.toString)})
}

def call2_k[T3](x: Int, k: Float => T3): T3 = {
    utilityFunction_k[T3](x, {f => k(f.toFloat)})
}

def mainFunction_k(x: Int, k:String => String):String = {
    call1_k[String](x.toString, v1 => {
      call2_k[String](x, v2 => {
          k(v1 + v2.toString)
      })  
    })
}

mainFunction_k(25, x => x)


defined function utilityFunction_k
defined function call1_k
defined function call2_k
defined function mainFunction_k
res27_4: String = "2727.0"

In general, it is always a good idea to build a CPS transformation assuming that the continuation can have any return type

# Trampolines

Trampolines go hand in hand with continuation passing style of writing programs. Let us quickly review the basic facts about CPS. 

    * we add an extra continuatoin arugment to every function call in the program 
    * we transform the program so that ll functoin calls happen at the tail position. 
    * Finally, we HOPE that the compiler / interpreter in all its goodness will optimatize the tail call away. 
    

## Fibonacci
We already saw the CPS version of Fibonacci

In [29]:
def fibonacci_k[T] (n: Int, k: Int => T): T = {
    if (n < 2)
        k(1)
    else 
        fibonacci_k(n-1, v1 => fibonacci_k(n-2, v2 => {k(v1 + v2)} ))
}

defined function fibonacci_k

In [30]:
fibonacci_k(10,print)
fibonacci_k(15,print)

89987